# Revisiting code

In [1]:
import numpy as np
import pandas as pd
import math
from siuba import *
from calitp import *
import intake
from shared_utils import geography_utils
import shared_utils
from plotnine import *
import altair as alt
import altair_saver
from shared_utils import altair_utils 
import cpi

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
#NB ONLY
import data_prep_refactored
import data_prep 
import agency_crosswalk
pd.options.display.float_format = "{:.2f}".format
pd.options.display.max_rows = 250

In [3]:
#Load in data
df_5311 = data_prep_refactored.load_grantprojects()
vehicles = data_prep_refactored.clean_vehicles_data()
organizations = data_prep_refactored.load_cleaned_organizations_data()
airtable = data_prep_refactored.load_airtable()

/home/jovyan/data-analyses/5311/data_prep_refactored.py:253: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## Test

In [4]:
test_aggregate = data_prep_refactored.aggregated_df()

/home/jovyan/data-analyses/5311/data_prep_refactored.py:253: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jovyan/data-analyses/5311/data_prep_refactored.py:355: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [5]:
test_aggregate.head()

,index,organization_name,reporter_type,fleet_size,ntd_id,itp_id,mpo_rtpa,GTFS,caltrans_district,planning_authority,Is_Agency_In_BC_Only_1_means_Yes,total_vehicles,average_age_of_fleet__in_years_,average_lifetime_miles_per_vehicle,Automobiles,Bus,Other,Train,Van,automobiles_door,bus_doors,van_doors,train_doors,doors_sum,_31_60,_16plus,_60plus,adjusted_closedoutbalance,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,adjusted_allocationamount,adjusted_expendedamount,adjusted_encumbered_amount,adjusted_activebalance,allocation_mean
0,0,Alpine County Community Development,None,No Info,None,None,None,None,None,None,1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,341366.00,293366.00,159901.31,181464.69,0,363613.49,173611.38,315613.45,190002.10,48766.57
1,1,Amador Regional Transit System,Rural Reporter,Medium,9R02-91000,None,None,Static OK_RT Incomplete,10 - Stockton,Amador County Transportation Commission,0,24.00,4.07,0.00,0.00,0.00,0.00,0.00,21.00,0.00,0.00,21.00,0.00,21.00,0.00,2.00,0.00,0.00,1518539.00,1279114.00,1067984.00,450555.00,0,1612119.29,1151645.28,1372694.08,460474.01,189817.38
2,2,Butte County Association of Governments,Full Reporter,Medium,90208,None,Butte County Association of Governments,Static OK_RT Incomplete,03 - Marysville,Butte County Association of Governments,0,53.00,6.23,200297.84,0.00,29.00,0.00,0.00,24.00,0.00,58.00,24.00,0.00,82.00,0.00,0.00,0.00,0.00,5886587.00,4820850.00,4362924.78,1523662.22,0,6267702.55,4719990.91,5201964.65,1547711.65,490548.92
3,3,Calaveras Transit Agency,Rural Reporter,Medium,9R02-99442,None,None,None,None,None,0,12.00,5.00,0.00,0.00,0.00,0.00,0.00,11.00,0.00,0.00,11.00,0.00,11.00,0.00,0.00,0.00,0.00,1688814.00,1914947.00,1081593.98,607220.02,0,1801548.92,1161866.84,2058384.96,639682.08,187646.00
4,4,"Central Contra Costa Transit Authority, dba: C...",Full Reporter,Large,90078,None,None,None,None,None,0,206.00,3.44,72814.41,0.00,125.00,0.00,0.00,62.00,0.00,250.00,62.00,0.00,312.00,0.00,2.00,0.00,0.00,50412.00,50412.00,0.00,50412.00,0,54399.71,0.00,54399.71,54399.71,50412.00


In [6]:
test_aggregate['reporter_type'].value_counts()

Rural Reporter        49
Full Reporter         26
None                   9
Reduced Reporter       2
Asset Subrecipient     1
Name: reporter_type, dtype: int64

In [7]:
test_aggregate['Is_Agency_In_BC_Only_1_means_Yes'].value_counts()

0    78
1     9
Name: Is_Agency_In_BC_Only_1_means_Yes, dtype: int64

In [8]:
test_aggregate.groupby(['fleet_size','organization_name']).agg({'total_vehicles':'sum'})

total_vehicles
fleet_size organization_name                                                 
Large      Central Contra Costa Transit Authority, dba: CO...          206.00
           County of Sonoma, dba: Sonoma County Transit                 94.00
           Eastern Sierra Transit Authority                             97.00
           Fresno County Rural Transit Agency                          178.00
           Kern Regional Transit                                        87.00
           Marin County Transit District                               116.00
           Monterey-Salinas Transit                                    196.00
           North County Transit District                               380.00
           Riverside Transit Agency                                    406.00
           San Diego Metropolitan Transit System                      1013.00
           San Joaquin Regional Transit District                       182.00
           San Luis Obispo Regional Transit Authority                   86.00
           San Mateo County Transit District                           578.00
           Santa Clara Valley Transportation Authority                1114.00
           Santa Cruz Metropolitan Transit District                    171.00
           SunLine Transit Agency                                      178.00
           Victor Valley Transit Authority                             400.00
Medium     Amador Regional Transit System                               24.00
           Butte County Association of Governments                      53.00
           Calaveras Transit Agency                                     12.00
           City of Dinuba                                               16.00
           City of Eureka, dba: Eureka Transit Service                  12.00
           City of Fairfield                                            65.00
           City of Porterville                                          42.00
           City of Ridgecrest                                           11.00
           City of Santa Maria                                          40.00
           City of Visalia                                              52.00
           Colusa County Transit Agency                                 13.00
           County of Placer, dba: Placer County Department...           70.00
           County of Siskiyou, dba: Siskiyou County Transit             15.00
           El Dorado County Transit Authority                           66.00
           Humboldt Transit Authority                                   37.00
           Imperial County Transportation Commission                    46.00
           Kings County Area Public Transit Agency                      34.00
           Lake Transit Authority                                       55.00
           Lassen Transit Service Agency                                25.00
           Livermore / Amador Valley Transit Authority                  84.00
           Madera County                                                16.00
           Mendocino Transit Authority                                  44.00
           Modoc Transportation Agency                                  14.00
           Morongo Basin Transit Authority                              44.00
           Mountain Area Regional Transit Authority                     34.00
           Napa Valley Transportation Authority                         78.00
           Palo Verde Valley Transit Agency                             11.00
           Redwood Coast Transit Authority                              19.00
           San Benito County Local Transportation Authority             22.00
           Tehama County                                                12.00
           Transit Joint Powers Authority for Merced County             81.00
           Tulare County Area Transit                                   34.00
           Tuolumne County Transit                                      30.00
       

In [9]:
original = data_prep.aggregated_merged_df()

/home/jovyan/data-analyses/5311/data_prep.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jovyan/data-analyses/5311/data_prep.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jovyan/data-analyses/5311/data_prep.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve

In [11]:
original['reporter_type'].value_counts()

Rural Reporter        49
Full Reporter         26
None                   9
Reduced Reporter       2
Asset Subrecipient     1
Name: reporter_type, dtype: int64

## Other Functions

In [13]:
def fleet_size_rating(df): 
    #FLEET SIZE 
    #First grabbing only one row for each agency into a new data frame 
    Fleet_size = df.groupby(['organization_name',]).agg({'total_vehicles':'max'}).reindex()
    #Get percentiles in objects for total vehicle.
    p75 = df.total_vehicles.quantile(0.75).astype(float)
    p25 = df.total_vehicles.quantile(0.25).astype(float)
    p50 = df.total_vehicles.quantile(0.50).astype(float)
    #Function for fleet size
    def fleet_size (row):
        if ((row.total_vehicles > 0) and (row.total_vehicles < p25)):
            return "Small"
        elif ((row.total_vehicles > p25) and (row.total_vehicles < p75)):
            return "Medium"
        elif ((row.total_vehicles > p50) and (row.total_vehicles > p75 )):
               return "Large"
        else:
            return "No Info"
    df["fleet_size"] = df.apply(lambda x: fleet_size(x), axis=1)
  
    return df

In [14]:
#Flag BlackCat only values
def blackcat_only(row):
    #If there are no values for GTFS, reporter type, and fleet size, then we can safely say
    #This organization is not registered by Cal ITP or NTD
    if ((row.GTFS == 'None') and (row.reporter_type == 'None') and row.fleet_size == 'No Info'):
        return "1"
    else:
        return "2"

In [15]:
crosswalk = {'City of Chowchilla ': 'City of Chowchilla, dba: Chowchilla Area Transit ',
     'City of Dinuba ':  'City of Dinuba',
     'Modoc Transportation Agency': 'Modoc Transportation Agency',
     'Butte County Association of Governments/ Butte Regional Transit': 'Butte County Association of Governments',
     'Calaveras County Public Works':  'Calaveras Transit Agency',
     'City of Escalon ':  'City of Escalon, dba: eTrans',
     'County of Mariposa':  'Mariposa County Transit, dba: Mari-Go',
     'County of Shasta Department of Public Works':  'County of Shasta Department of Public Works',
     'County of Siskiyou': 'County of Siskiyou, dba: Siskiyou County Transit',
     'County of Tulare': 'Tulare County Area Transit',
     'Eureka Transit Service':  'City of Eureka, dba: Eureka Transit Service',
     'Kern Regional Transit':  'Kern Regional Transit',
     'Livermore Amador Valley Transit Authority':  'Livermore / Amador Valley Transit Authority',
     'Placer County Public Works (TART & PCT)': 'County of Placer, dba: Placer County Department of Public Works',
     'Plumas County Transportation Commission': 'Plumas County Transportation Commission',
     'San Luis Obispo Regional Transit Authority':  'San Luis Obispo Regional Transit Authority',
     'Sonoma County Transit':  'County of Sonoma, dba: Sonoma County Transit',
     'Sunline Transit Agency':  'SunLine Transit Agency',
     'Tehama County Transit Agency': 'Tehama County',
     'Trinity County Department of Transportation ':  'Trinity County',
     'Tuolumne County Transit Agency (TCTA)':  'Tuolumne County Transit',
     'Amador Transit':  'Amador Regional Transit System',
     'City of Corcoran - Corcoran Area Transit':  'City of Corcoran, dba: Corcoran Area Transit',
     'Yosemite Area Regional Transportation System ':  'Yosemite Area Regional Transportation System',
     'County Connection (Central Contra Costa Transit Authority)': 'Central Contra Costa Transit Authority, dba: COUNTY CONNECTION',
     'Calaveras Transit Agency ': 'Calaveras Transit Agency'}

In [16]:
# Inflation table
def inflation_table(base_year):
    cpi.update()
    series_df = cpi.series.get(area="U.S. city average").to_dataframe()
    inflation_df = (series_df[series_df.year >= 2008]
           .pivot_table(index='year', values='value', aggfunc='mean')
           .reset_index()
          )
    denominator = inflation_df.value.loc[inflation_df.year==base_year].iloc[0]

    inflation_df = inflation_df.assign(
        inflation = inflation_df.value.divide(denominator)
    )
    
    return inflation_df

def adjust_prices(df):
    
    cols =  ["allocationamount",
             "encumbered_amount",
             "expendedamount",
             "activebalance",
             "closedoutbalance"]
    
    ##get cpi table 
    cpi = inflation_table(2021)
    cpi.update
    cpi = (cpi>>select(_.year, _.value))
    cpi_dict = dict(zip(cpi['year'], cpi['value']))
    
    
    for col in cols:
        multiplier = df["project_year"].map(cpi_dict)  
    
        ##using 270.97 for 2021 dollars
        df[f"adjusted_{col}"] = ((df[col] * 270.97) / multiplier)
    return df


## Function: Merge NTD Vehicles with  Airtable

In [17]:
'''
Merge NTD Vehicles data set with GTFS info from Airtable
'''
def ntd_airtable_merge():
    #Importing the dataframes: organizations is Cal ITP's Airtable
    #Vehicles is from NTD's vechicles dataset
    organizations = data_prep_refactored.load_cleaned_organizations_data() #Delete data_prep_refactored once moving to the scripts
    vehicles = data_prep_refactored.clean_vehicles_data() 
    #merge the 2 datasets on the left, since there are many more entries on the left
    m1 = pd.merge(vehicles, organizations,  how='left', on=['ntd_id'])
    return m1

## Function: Merge M1 with 5311 Info Black Cat

In [18]:
'''
Merge m1 with 5311 info from BlackCat
'''
def ntd_airtable_5311_merge():
    m1 =  ntd_airtable_merge()
    df_5311 = data_prep_refactored.load_grantprojects()
    #FIRST MERGE: 
    #5311 info from Black Cat on the left.
    m2 = (pd.merge(df_5311, m1,  how='left', left_on=['organization_name'], 
                      right_on=['name'], indicator=True)
            )
    #Some matches failed:subset out a df with left only matches
    Left_only = m2[(m2._merge.str.contains("left_only", case= False))] 
    
    #Take organizations left and make it into a list
    Left_orgs = Left_only['organization_name'].drop_duplicates().tolist()
    
    #Delete  left only matches from original df 
    m2 = m2[~m2.organization_name.isin(Left_orgs)]
    
    #SECOND MERGE:
    #As we deleted organizations that were "left only",
    #make a data frame with these values & replace their names with ones in NTD data
    fail = df_5311[df_5311.organization_name.isin(Left_orgs)]
    
    #replacing organization names from Black Cat with agency names from m1  
    fail['organization_name'].replace(crosswalk, inplace= True)
    
    #Merging the "failed" dataframe with m1 (NTD and GTFS Airtable info) 
    m3 = pd.merge(fail, m1,  how='left', left_on=['organization_name'], right_on=['agency'])
    
    #Concat: 
    m4 = pd.concat([m2, m3])
    return m4
    

In [19]:
ntd_gtfs_bc = ntd_airtable_5311_merge()

/home/jovyan/data-analyses/5311/data_prep_refactored.py:253: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_1567/2781274361.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## Function: Final DF 
* Cleaning up merged df & adding metric flags 

In [20]:
def final_df():
    df1 = ntd_airtable_5311_merge()
    airtable2 = data_prep_refactored.load_airtable()
    #Manually replace Klamath
    #Klamath does not appear in NTD data so we missed it when we merged NTD & Cal ITP on NTD ID
    df1.loc[(df1['organization_name'] == 'Klamath Trinity Non-Emergency Transportation\u200b'), "itp_id"] = "436"
    df1.loc[(df1['organization_name'] == 'Klamath Trinity Non-Emergency Transportation\u200b'), "gtfs_schedule_status"] = "needed"
    
    #Change ITP ID ID to be floats & 0 so parquet will work
    df1['itp_id'] = df1['itp_id'].fillna(0).str.replace("'", "")
    #df1.loc[(df1['itp_id'] == '436'), "itp_id"] = 436
    #Del & rename columns
    df2 = df1.drop(columns=['name','gtfs_schedule_status','agency'])
    
    #Apply functions 
    #Call inflation function to add $ columns with adjusted values for inflation 
    df2 = adjust_prices(df1)
    #Apply fleet size() function
    df2 = fleet_size_rating(df1)
    #Merge df1 with the new airtable stuff 
    final = pd.merge(df1, airtable2, how='left', left_on='name', right_on='name')
    
    #Concatenate the two GTFS cols together into one column 
    final["GTFS"] = final["gtfs_static_status"] + '_' + final["gtfs_realtime_status"]
    
    #Drop old columns
    final = final.drop(columns = ['gtfs_static_status','gtfs_realtime_status','_merge'])
    #Fill NA by data types 
    final.fillna(final.dtypes.replace({'float64': 0.0, 'object': 'None'}), inplace=True)
   
    #Apply Black Cat only function 
    final["Is_Agency_In_BC_Only_1_means_Yes"] = final.apply(lambda x: blackcat_only(x), axis=1)
    
    return final


In [21]:
ntd_gtfs_bc = final_df()

/home/jovyan/data-analyses/5311/data_prep_refactored.py:253: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_1567/2781274361.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [22]:
ntd_gtfs_bc['Is_Agency_In_BC_Only_1_means_Yes'].value_counts()

2    743
1     55
Name: Is_Agency_In_BC_Only_1_means_Yes, dtype: int64

In [23]:
ntd_gtfs_bc.head(2)

,grant_fiscal_year,funding_program,grant_number,project_year,organization_name,upin,description,ali,contract_number,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,project_status,agency,ntd_id,reporter_type,_0_9,_10_12,_13_15,_16_20,_21_25,_26_30,_31_60,_60plus,total_vehicles,average_age_of_fleet__in_years_,average_lifetime_miles_per_vehicle,sum_15plus,Automobiles,Bus,Other,Service,Train,Van,van_doors,automobiles_door,bus_doors,train_doors,doors_sum,name,itp_id,gtfs_schedule_status,adjusted_allocationamount,adjusted_encumbered_amount,adjusted_expendedamount,adjusted_activebalance,adjusted_closedoutbalance,fleet_size,caltrans_district,mpo_rtpa,planning_authority,GTFS,Is_Agency_In_BC_Only_1_means_Yes
0,2011,Section 5311,CA-18-X047 | 0012000083,2016,Madera County,BCG0000283,Buy <30-Ft Bus For Expansion,111304,64BC17-00408,110663.00,110663.00,101352.02,9310.98,0,Open,"Madera County, dba: Madera County Connection T...",9R02-91005,Rural Reporter,16.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,16.00,5.46,0.00,0.00,0.00,0.00,0.00,0.00,0.00,16.00,16.00,0.00,0.00,0.00,16.00,Madera County,None,"ok,needed,needed",124939.41,124939.41,114427.24,10512.17,0.00,Medium,06 - Fresno,None,None,Static OK_RT OK,2
1,2011,Section 5311,CA-18-X047 | 0012000083,2016,Madera County,BCG0000284,Purchase Replacement Van,111215,64BC17-00408,20643.00,44265.00,20643.00,0.00,0,Open,"Madera County, dba: Madera County Connection T...",9R02-91005,Rural Reporter,16.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,16.00,5.46,0.00,0.00,0.00,0.00,0.00,0.00,0.00,16.00,16.00,0.00,0.00,0.00,16.00,Madera County,None,"ok,needed,needed",23306.11,49975.54,23306.11,0.00,0.00,Medium,06 - Fresno,None,None,Static OK_RT OK,2


In [24]:
ntd_gtfs_bc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 798 entries, 0 to 797
Data columns (total 55 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   grant_fiscal_year                   798 non-null    int64  
 1   funding_program                     798 non-null    object 
 2   grant_number                        798 non-null    object 
 3   project_year                        798 non-null    int64  
 4   organization_name                   798 non-null    object 
 5   upin                                798 non-null    object 
 6   description                         798 non-null    object 
 7   ali                                 798 non-null    object 
 8   contract_number                     798 non-null    object 
 9   allocationamount                    798 non-null    float64
 10  encumbered_amount                   798 non-null    float64
 11  expendedamount                      798 non-n

## Aggregate

In [25]:
      
def aggregated_df(): 
    #Read original df & airtable
    df = final_df()
    
    #Aggregate all vehicles 16+ above into one col 
    column_names = ['_16_20', '_21_25', '_26_30', '_31_60']
    df['_16plus']= df[column_names].sum(axis=1)
    
    #duplicate allocationamount col so we can get the mean an organiztaion received.
    df['allocation_mean'] = df['allocationamount']
    #https://stackoverflow.com/questions/67717440/use-a-list-of-column-names-in-groupby-agg
    df = (df.groupby(['organization_name',
                     'reporter_type', 
                     'fleet_size', 
                     'ntd_id', 
                     'itp_id','mpo_rtpa','GTFS','caltrans_district','planning_authority'], as_index=False)
    .agg({**{e:'max' for e in max_cols}, **{e:'sum' for e in sum_cols}, **{e: 'mean' for e in mean_cols}}).reset_index())
   
    return df 
    

In [26]:
agg_df_for_analysis = aggregated_df()

/home/jovyan/data-analyses/5311/data_prep_refactored.py:253: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_1567/2781274361.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


NameError: name 'max_cols' is not defined

In [ ]:
agg_df_for_analysis.head(3)

## Double Check

In [ ]:
original.shape

In [ ]:
original.head(2)

In [ ]:

set(original.columns).difference(set(agg_df_for_analysis.columns))

In [ ]:
org_original = set(original.organization_name.unique().tolist())
org_script = set(agg_df_for_analysis.organization_name.unique().tolist())

In [ ]:
org_original-org_script

In [ ]:
org_script-org_original

In [ ]:
original['GTFS_Status'].value_counts()

In [ ]:
original['Is_Agency_In_BC_Only_1_means_Yes'].value_counts()

In [ ]:
agg_df_for_analysis['GTFS'].value_counts()

In [ ]:
agg_df_for_analysis['Is_Agency_In_BC_Only_1_means_Yes'].value_counts()

In [ ]:
agg_df_for_analysis.groupby(['caltrans_district', 'organization_name','GTFS']).agg({'allocation_mean':'max'})

In [ ]:
agg_df_for_analysis['organization_name'].nunique()

In [ ]:
def pick_column_and_aggregate(df1, df2, col):
    if df1[col].sum() == df2[col].sum():
        print("PASS")
    else:
        print(f"{col}: FAIL")

In [ ]:
my_columns = ['allocationamount',
       'encumbered_amount', 'expendedamount', 'activebalance',
       'closedoutbalance', 'adjusted_allocationamount',
       'adjusted_expendedamount', 'adjusted_encumbered_amount',
       'adjusted_activebalance', 'adjusted_closedoutbalance', 'total_vehicles',]

In [ ]:
for c in my_columns:
    pick_column_and_aggregate(agg_df_for_analysis, original, c)